In [ ]:
# CELL 1: Install Required Libraries

!pip install -q roboflow ultralytics scikit-learn pyyaml tqdm opencv-python

In [ ]:
# CELL 2: Prepare CSV with correct column names

import pandas as pd

csv_path = "/content/drive/MyDrive/MADE_Unmade/data_stage1/test.csv"
df = pd.read_csv(csv_path)

# Rename column if needed
if "pred_label" in df.columns:
    df = df.rename(columns={"pred_label": "label"})

df.to_csv(csv_path, index=False)
print(f"CSV ready with columns: {df.columns}")

In [ ]:
# CELL 3: Run Robustness Test

OUT_DIR = "/content/drive/MyDrive/robustness_test/results/robustness"
!mkdir -p "$OUT_DIR"

!python -m src.robustness.run_robustness \
    --config /content/drive/MyDrive/robustness_test/configs/robustness.yaml \
    --stage1_config /content/drive/MyDrive/robustness_test/configs/stage1_binary.yaml \
    --ckpt /content/drive/MyDrive/project/runs/stage1_binary/best_stage1.pt \
    --data_root /content/drive/MyDrive/MADE_Unmade/data_stage1 \
    --val_csv test.csv \
    --img_dir images/test \
    --out_dir "$OUT_DIR"

In [ ]:
# CELL 4: View Robustness Report

import json

REPORT_PATH = "/content/drive/MyDrive/robustness_test/results/robustness/robustness_report.json"

with open(REPORT_PATH, 'r') as f:
    report = json.load(f)

print("Full Robustness Report:")
print(json.dumps(report, indent=2))

In [ ]:
# CELL 5: Visualize Corruptions

import os
import sys
import matplotlib.pyplot as plt
from PIL import Image
import random

# Add robustness module path
sys.path.append('/content/drive/MyDrive/robustness_test')
from src.robustness.corruptions import brightness, gaussian_noise, gaussian_blur, contrast

# Paths
DATA_ROOT = "/content/drive/MyDrive/MADE_Unmade/data_stage1"
IMG_DIR = os.path.join(DATA_ROOT, "images/test")
OUT_DIR = "/content/drive/MyDrive/robustness_test/results/robustness"
os.makedirs(OUT_DIR, exist_ok=True)

# Load a sample image
sample_imgs = [f for f in os.listdir(IMG_DIR) if f.lower().endswith('.jpg')]
sample_img_path = os.path.join(IMG_DIR, random.choice(sample_imgs))
original = Image.open(sample_img_path)

# Apply corruptions
corrupted_images = {
    "Original": original,
    "Brightness -20%": brightness(original, 0.2),
    "Brightness -50%": brightness(original, 0.5),
    "Gaussian Noise 2%": gaussian_noise(original, 0.02),
    "Gaussian Noise 5%": gaussian_noise(original, 0.05),
    "Gaussian Blur r=1": gaussian_blur(original, 1.0),
    "Gaussian Blur r=2": gaussian_blur(original, 2.0),
    "Contrast -20%": contrast(original, 0.2),
}

# Plot
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for idx, (title, img) in enumerate(corrupted_images.items()):
    axes[idx].imshow(img)
    axes[idx].set_title(title, fontsize=10)
    axes[idx].axis('off')

plt.tight_layout()
plt_path = os.path.join(OUT_DIR, 'corruption_samples.png')
plt.savefig(plt_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"Corruption visualization saved to: {plt_path}")

In [ ]:
# CELL 6: Visualize Model Predictions on Corrupted Images

import os
import sys
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import torch
import cv2
import pandas as pd
import random

# Add paths
sys.path.append('/content/drive/MyDrive/robustness_test')
from src.robustness.corruptions import brightness, gaussian_noise, gaussian_blur, contrast
from src.common.utils import get_device
from src.common.transforms import get_eval_transforms
from src.stage1_binary.model import build_model

# Paths
DATA_ROOT = "/content/drive/MyDrive/MADE_Unmade/data_stage1"
IMG_DIR = os.path.join(DATA_ROOT, "images/test")
OUT_DIR = "/content/drive/MyDrive/robustness_test/results/robustness"
CKPT_PATH = "/content/drive/MyDrive/stage1_results/stage1_binary/best_stage1.pt"
CONFIG_PATH = "/content/drive/MyDrive/robustness_test/configs/stage1_binary.yaml"

# Load config
import yaml
with open(CONFIG_PATH, 'r') as f:
    cfg = yaml.safe_load(f)

# Load model
device = get_device()
model = build_model(
    backbone=cfg["model"]["backbone"],
    num_classes=int(cfg["model"]["num_classes"]),
    pretrained=False,
    dropout=float(cfg["model"]["dropout"]),
).to(device)

ckpt = torch.load(CKPT_PATH, map_location="cpu")
model.load_state_dict(ckpt["model"])
model.eval()

# Get transforms
transforms = get_eval_transforms(int(cfg["model"]["img_size"]))

# Label mapping
label_names = {0: 'Made', 1: 'Unmade'}


def predict_image(img_pil, model, transforms, device):
    """Predict on a PIL image and return label + confidence."""
    img_rgb = np.array(img_pil)
    x = transforms(image=img_rgb)["image"].unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1)
        conf, pred = torch.max(probs, dim=1)

    pred_label = int(pred.cpu().numpy()[0])
    confidence = float(conf.cpu().numpy()[0])

    return pred_label, confidence


# Select multiple sample images (mix of Made and Unmade if possible)
sample_imgs = [f for f in os.listdir(IMG_DIR) if f.lower().endswith('.jpg')]
num_samples = min(3, len(sample_imgs))
selected_imgs = random.sample(sample_imgs, num_samples)

# For each sample image
for img_idx, img_name in enumerate(selected_imgs):
    print(f"\n{'='*80}")
    print(f"Processing Image {img_idx+1}/{num_samples}: {img_name}")
    print(f"{'='*80}")

    img_path = os.path.join(IMG_DIR, img_name)
    original = Image.open(img_path)

    # Define corruptions to test
    corruptions_to_test = [
        ("Original", None, 0),
        ("Brightness -20%", brightness, 0.2),
        ("Brightness -50%", brightness, 0.5),
        ("Gaussian Noise 2%", gaussian_noise, 0.02),
        ("Gaussian Noise 5%", gaussian_noise, 0.05),
        ("Gaussian Blur r=1", gaussian_blur, 1.0),
        ("Gaussian Blur r=2", gaussian_blur, 2.0),
        ("Contrast -20%", contrast, 0.2),
    ]

    # Predict on each corruption
    results = []
    corrupted_images = []

    for title, corr_fn, magnitude in corruptions_to_test:
        if corr_fn is None:
            img_corrupted = original
        else:
            img_corrupted = corr_fn(original, magnitude)

        pred_label, confidence = predict_image(img_corrupted, model, transforms, device)

        results.append({
            'title': title,
            'pred_label': pred_label,
            'pred_name': label_names[pred_label],
            'confidence': confidence
        })
        corrupted_images.append(img_corrupted)

        print(f"  {title:20s} → Pred: {label_names[pred_label]:7s} (conf: {confidence:.3f})")

    # Visualize
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()

    for idx, (img, result) in enumerate(zip(corrupted_images, results)):
        axes[idx].imshow(img)

        # Color code: green if high confidence, red if low
        color = 'green' if result['confidence'] > 0.8 else 'orange' if result['confidence'] > 0.6 else 'red'

        title_text = f"{result['title']}\nPred: {result['pred_name']} ({result['confidence']:.2f})"
        axes[idx].set_title(title_text, fontsize=9, color=color, fontweight='bold')
        axes[idx].axis('off')

    plt.suptitle(f"Robustness Test: {img_name}", fontsize=14, fontweight='bold')
    plt.tight_layout()

    # Save
    save_path = os.path.join(OUT_DIR, f'predictions_sample_{img_idx+1}.png')
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()

    print(f"\n  Saved to: {save_path}")


print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print("All prediction visualizations saved to:")
print(f"  {OUT_DIR}/predictions_sample_*.png")
print("\nColor coding:")
print("  🟢 Green:  High confidence (>0.8)")
print("  🟠 Orange: Medium confidence (0.6-0.8)")
print("  🔴 Red:    Low confidence (<0.6)")